In [1]:
from dotenv import load_dotenv
import os
import base64
from requests import post, get
import json

load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

In [34]:
# function that gets authentication token according to spotify format/standards
def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization" : "Basic " + auth_base64,
        "Content-Type" : "application/x-www-form-urlencoded"
    }
    data = {"grant_type" : "client_credentials", 'scope': 'user-top-read'}
    result = post(url, headers = headers, data = data, verify = True)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    # print(json_result['token_type'])
    return token

In [19]:
def get_auth_header(token):
    return {"Authorization" : "Bearer " + token}

In [20]:
# function that retrieves the first artist that matches the given name
def search_for_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={artist_name}&type=artist&limit=1"

    query_url = url + query
    result = get(query_url, headers = headers)
    json_result = json.loads(result.content)['artists']['items']
    if len(json_result) == 0:
        print("No artist with this name exists...")
        return None
    return json_result[0]

In [21]:
def get_songs_by_artist(token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks?country=US"
    headers = get_auth_header(token)
    result = get(url, headers = headers)
    json_result = json.loads(result.content)['tracks']
    return json_result

In [32]:
def get_top_artists(token):
    url = "https://api.spotify.com/v1/me/top/artists"
    headers = get_auth_header(token)
    result = get(url, headers = headers)
    print(result)
    # json_result = json.loads(result.content)
    # return json_result

In [35]:
token = get_token()
token

'BQCswTqOFqei--Pjzv4W2r9C7Lt79wwW5qj-RQNlmUWDyNqiwCK7xjX6GQ9v7bfyTJ0MCG6q_qqUR6dWxeyYTc7GaiRcY7UVq_ox2H6233ZLgmddpkY7hrODh4E'

In [36]:
result = search_for_artist(token, "ACDC")
artist_id = result['id']
artist_id

'711MCceyCBcFnzjGY4Q7Un'

In [37]:
songs = get_songs_by_artist(token, artist_id)
for idx, song in enumerate(songs):
    print(f"{idx + 1}. {song['name']}")

1. Highway to Hell
2. Thunderstruck
3. Back In Black
4. You Shook Me All Night Long
5. T.N.T.
6. Hells Bells
7. Shoot to Thrill
8. Dirty Deeds Done Dirt Cheap
9. It's a Long Way to the Top (If You Wanna Rock 'N' Roll)
10. Rock N Roll Train


In [38]:
top = get_top_artists(token)
top

<Response [500]>
